<a href="https://colab.research.google.com/github/princy2352/ML-Projects/blob/main/Llama3_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print(dataset)
print(dataset["train"][0])
print(dataset["train"].features)
print(len(dataset["train"]))


DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})
{'id': 5097, 'domain': 'forestry', 'domain_description': 'Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.', 'sql_complexity': 'single join', 'sql_complexity_description': 'only one join (specify inner, outer, cross)', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'What is the total vol

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

dataset = load_dataset("gretelai/synthetic_text_to_sql")

# Function to convert a single example to the Llama 3.1 chat template format
def convert_to_llama_format(example):

    system_content = "You are a SQL assistant that helps users write SQL queries based on their requirements and database schema."

    # User message combining SQL prompt and context
    user_content = f"SQL Prompt: {example['sql_prompt']}\n\nContext:\n{example['sql_context']}"

    # Assistant message combining SQL query and explanation
    assistant_content = f"SQL Query:\n```sql\n{example['sql']}\n```\n\nExplanation: {example['sql_explanation']}"

    # Format the conversation according to the Llama 3.1 chat template
    formatted_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_content}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_content}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{assistant_content}<|eot_id|>"

    # Return the formatted text
    return {
        "text": formatted_text,
        "system_content": system_content,
        "user_content": user_content,
        "assistant_content": assistant_content
    }



In [ ]:
# Process the training set
train_data = [convert_to_llama_format(example) for example in dataset["train"]]
train_dataset = Dataset.from_list(train_data)

# Process the test set
test_data = [convert_to_llama_format(example) for example in dataset["test"]]
test_dataset = Dataset.from_list(test_data)

# Create a new DatasetDict
llama_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [ ]:
# Display information about the transformed dataset
print("Transformed dataset structure:")
print(llama_dataset)
print("\nSample entry from the train split:")
print(llama_dataset["train"][0]["text"][:500] + "...") # Print first 500 chars of the first example
print(f"\nNumber of examples - Train: {len(llama_dataset['train'])}, Test: {len(llama_dataset['test'])}")

# Save the dataset
llama_dataset.save_to_disk("llama31_finetuning_dataset")
print("\nDataset saved to 'llama31_finetuning_dataset' directory")


Transformed dataset structure:
DatasetDict({
    train: Dataset({
        features: ['text', 'system_content', 'user_content', 'assistant_content'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['text', 'system_content', 'user_content', 'assistant_content'],
        num_rows: 5851
    })
})

Sample entry from the train split:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a SQL assistant that helps users write SQL queries based on their requirements and database schema.<|eot_id|><|start_header_id|>user<|end_header_id|>

SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?

Context:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith...

Number of examples - Train: 100000, Test: 5851


Saving the dataset (0/1 shards):   0%|          | 0/100000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5851 [00:00<?, ? examples/s]


Dataset saved to 'llama31_finetuning_dataset' directory


In [ ]:
from datasets import load_from_disk
loaded_dataset = load_from_disk("llama31_finetuning_dataset")
print("\nVerified that the dataset can be loaded successfully")


Verified that the dataset can be loaded successfully


In [ ]:
loaded_dataset=loaded_dataset['train']

<a name="Train"></a>
### Train the model


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = loaded_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a SQL assistant that helps users write SQL queries based on their requirements and database schema.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSQL Prompt: What is the total spending on humanitarian assistance by the European Union in the last 3 years?\n\nContext:\nCREATE SCHEMA if not exists defense; CREATE TABLE if not exists eu_humanitarian_assistance (id INT PRIMARY KEY, year INT, spending INT); INSERT INTO defense.eu_humanitarian_assistance (id, year, spending) VALUES (1, 2019, 1500), (2, 2020, 1800), (3, 2021, 2100);<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL Query:\n```sql\nSELECT SUM(spending) FROM defense.eu_humanitarian_assistance WHERE year BETWEEN 2019 AND 2021;\n```\n\nExplanation: This SQL query calculates the total spending on humanitarian assistance by the European Union in the last 3 years. It uses the SUM() aggregation function to find the total spending and WHERE claus

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                        \n\nSQL Query:\n```sql\nSELECT SUM(spending) FROM defense.eu_humanitarian_assistance WHERE year BETWEEN 2019 AND 2021;\n```\n\nExplanation: This SQL query calculates the total spending on humanitarian assistance by the European Union in the last 3 years. It uses the SUM() aggregation function to find the total spending and WHERE clause to filter the result by year.<|eot_id|>'

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/4,582,543,360 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.048100
2,1.008100
3,1.010400
4,0.924300
5,0.681500
6,0.724000
7,0.669100
8,0.572400
9,0.575500
10,0.604700


<a name="Inference"></a>
### Inference


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [{"role": "system", "content": "You are a SQL assistant that helps users write SQL queries based on their requirements and database schema."}, {"role": "user", "content": "SQL Prompt: What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?\n\nContext:\nCREATE TABLE creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT); INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'ApplicationY', 'North America', 0.91), (3, 'ApplicationZ', 'Europe', 0.84), (4, 'ApplicationAA', 'North America', 0.93), (5, 'ApplicationAB', 'Europe', 0.89);"},
            ]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a SQL assistant that helps users write SQL queries based on their requirements and database schema.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSQL Prompt: What is the average explainability score of creative AI applications in 'Europe' and 'North America' in the 'creative_ai' table?\n\nContext:\nCREATE TABLE creative_ai (application_id INT, name TEXT, region TEXT, explainability_score FLOAT); INSERT INTO creative_ai (application_id, name, region, explainability_score) VALUES (1, 'ApplicationX', 'Europe', 0.87), (2, 'ApplicationY', 'North America', 0.91), (3, 'ApplicationZ', 'Europe', 0.84), (4, 'ApplicationAA', 'North America', 0.93), (5, 'ApplicationAB', 'Europe', 0.89);<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL Query:\n```sql\nSELECT AVG(explainability_score) FROM creative_ai WHERE region IN ('Europe', 'North America

<a name="Save"></a>
### Saving finetuned models


In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')